In [0]:
import os, zipfile
import pandas as pd
import numpy as np
from keras.layers import Dense, Input, GRU, Embedding, Dropout, Bidirectional, LSTM, merge, add, GlobalAveragePooling1D, \
SpatialDropout1D, Flatten, Activation, RepeatVector, Permute, multiply, CuDNNLSTM, concatenate, Layer, GlobalMaxPooling1D, TimeDistributed
from keras.models import Model, Sequential
from keras.optimizers import Adam
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, LearningRateScheduler
from tqdm import tqdm_notebook
import tensorflow as tf
from keras import backend as K
from sklearn.utils import shuffle
import string
from keras.initializers import he_normal
import nltk
from nltk.corpus import stopwords

Using TensorFlow backend.


In [0]:
! mkdir ~/.kaggle/ && echo '{"username":"nashikunov","key":""}' > ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle/’: File exists


In [0]:
import kaggle

In [0]:
kaggle.api.competition_download_files('iad-deep-learning-sentiment')

In [0]:
! kaggle competitions download -c iad-deep-learning-sentiment

x_train.txt.zip: Skipping, found more recently modified local copy (use --force to force download)
random_prediction.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
x_test.txt.zip: Skipping, found more recently modified local copy (use --force to force download)
y_train.csv.zip: Skipping, found more recently modified local copy (use --force to force download)


In [0]:
!ls

glove.42B.300d.txt		 random_prediction.csv.zip  x_train.txt.zip
glove.42B.300d.zip		 sample_data		    y_train.csv
glove.42B.300d.zip.1		 x_test.txt		    y_train.csv.zip
iad-deep-learning-sentiment.zip  x_test.txt.zip
random_prediction.csv		 x_train.txt


In [0]:
! unzip x_train.txt.zip
! unzip y_train.csv.zip
! unzip x_test.txt.zip

In [0]:
! unzip random_prediction.csv.zip

In [0]:
X = pd.read_csv('x_train.txt', delimiter= "\r\t", header=None)
y = pd.read_csv('y_train.csv')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [0]:
data = X.join(y).drop('Id', axis=1)

In [0]:
del X, y

In [0]:
data.head()

,0,Probability
0,Stuning even for the non-gamer: This sound tra...,1.0
1,The best soundtrack ever to anything.: I'm rea...,1.0
2,Amazing!: This soundtrack is my favorite music...,1.0
3,Excellent Soundtrack: I truly like this soundt...,1.0
4,"Remember, Pull Your Jaw Off The Floor After He...",1.0


## Outliers

In [0]:
data['count_word']=data[0].apply(lambda x: len(str(x).split()))
data['count_unique_word']=data[0].apply(lambda x: len(set(str(x).split())))
data['word_unique_percent']=data['count_unique_word']*100/data['count_word']

In [0]:
data.drop(data[data.count_word <= np.percentile(data.count_word, 1)].index, inplace=True)
data.drop(data[data.count_word >= np.percentile(data.count_word, 99)].index, inplace=True)

In [0]:
data.drop(data[data.word_unique_percent < 50].index, inplace=True)

In [0]:
data.shape

(3516903, 5)

## Punctuation

In [0]:
X_test = pd.read_csv('x_test.txt', delimiter= "\r\t", header=None)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [0]:
data.shape

(3516903, 5)

In [0]:
data = pd.concat([data, X_test], ignore_index=True)

In [0]:
data.shape

(3916903, 5)

In [0]:
punctuation = list(string.punctuation)
def delete_punctuation(x):
    return ''.join([a if a not in punctuation + ['\n'] else ' ' for a in x])


In [0]:
data[0] = data[0].apply(delete_punctuation)
print('DONE')
data[0] = data[0].apply(lambda x: x.lower())

DONE


In [0]:
data.head()

,0,Probability,count_unique_word,count_word,word_unique_percent
0,stuning even for the non gamer this sound tra...,1.0,61.0,80.0,76.250000
1,the best soundtrack ever to anything i m rea...,1.0,73.0,97.0,75.257732
2,amazing this soundtrack is my favorite music...,1.0,101.0,129.0,78.294574
3,excellent soundtrack i truly like this soundt...,1.0,87.0,118.0,73.728814
4,remember pull your jaw off the floor after he...,1.0,71.0,87.0,81.609195


Не стал делать

In [0]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
stop_words = list(set(stopwords.words('english')) - set(['no', 'not', 'nor']))
def delete_stop(x):
    return ' '.join([item for item in x.split() if item not in stop_words])

## Stop words

In [0]:
data[0] = data[0].apply(delete_stop)

In [0]:
data.head()

,0,Probability,count_unique_word,count_word,word_unique_percent
0,stuning even non gamer sound track beautiful p...,1.0,61.0,80.0,76.250000
1,best soundtrack ever anything reading lot revi...,1.0,73.0,97.0,75.257732
2,amazing soundtrack favorite music time hands i...,1.0,101.0,129.0,78.294574
3,excellent soundtrack truly like soundtrack enj...,1.0,87.0,118.0,73.728814
4,remember pull jaw floor hearing played game kn...,1.0,71.0,87.0,81.609195


In [0]:
data['count_word_clean']=data[0].apply(lambda x: len(str(x).split()))
data['count_unique_word_clean']=data[0].apply(lambda x: len(set(str(x).split())))

In [0]:
data.head()

,0,Probability,count_unique_word,count_word,word_unique_percent
0,stuning even for the non gamer this sound tra...,1.0,61.0,80.0,76.250000
1,the best soundtrack ever to anything i m rea...,1.0,73.0,97.0,75.257732
2,amazing this soundtrack is my favorite music...,1.0,101.0,129.0,78.294574
3,excellent soundtrack i truly like this soundt...,1.0,87.0,118.0,73.728814
4,remember pull your jaw off the floor after he...,1.0,71.0,87.0,81.609195


In [0]:
#data = data.sample(frac=1).reset_index(drop=True)

In [0]:
!wget http://nlp.stanford.edu/data/wordvecs/glove.42B.300d.zip

--2019-12-23 19:54:45--  http://nlp.stanford.edu/data/wordvecs/glove.42B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/wordvecs/glove.42B.300d.zip [following]
--2019-12-23 19:54:45--  https://nlp.stanford.edu/data/wordvecs/glove.42B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/wordvecs/glove.42B.300d.zip [following]
--2019-12-23 19:54:45--  http://downloads.cs.stanford.edu/nlp/data/wordvecs/glove.42B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Le

In [0]:
file_name = os.path.abspath('./glove.42B.300d.zip') # get full path of files
zip_ref = zipfile.ZipFile(file_name) # create zipfile object
zip_ref.extractall('./') # extract file to dir
zip_ref.close() # close file
os.remove(file_name) # delete zipped file

BadZipFile: ignored

In [0]:
f = open('glove.42B.300d.txt')
embeddings_index = dict() 
for line in tqdm_notebook(f):
    values = line.split(' ')
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [0]:
embed_size = 300 # how big is each word vector
max_features = 70000 # how many unique words to use (i.e num rows in embedding vector) 60к
maxlen = 200 # max number of words in a comment to use 80

list_sentences_train = data[0].values
y = data.Probability.values

tokenizer = Tokenizer(num_words=max_features, )
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
x_train_pad = pad_sequences(list_tokenized_train, maxlen=maxlen)

In [0]:
all_embs = np.stack(embeddings_index.values())
emb_mean, emb_std = all_embs.mean(), all_embs.std()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if self.run_code(code, result):


In [0]:
word_index = tokenizer.word_index
unknown_words = set()
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: 
      continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: 
      embedding_matrix[i] = embedding_vector
    else: 
      unknown_words.add(word)

In [0]:
len(unknown_words)

168

In [0]:
np.shape(embedding_matrix)

(70000, 300)

In [0]:
def reset_tf_session():
    curr_session = tf.get_default_session()
    if curr_session is not None:
        curr_session.close()
    K.clear_session()
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    s = tf.InteractiveSession(config=config)
    K.set_session(s)
    return s

In [0]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [0]:
len(y[y >= 0])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in greater_equal
  """Entry point for launching an IPython kernel.


3516903

In [0]:
y[:3516903]

array([1., 1., 1., ..., 0., 0., 1.])

In [0]:
data_train = pd.DataFrame(x_train_pad[:3516903])
data_train['y'] = y[:3516903]
data_train = shuffle(data_train)

In [0]:
y = data_train.y.values
data_train.drop('y', axis=1, inplace=True)
X = data_train.values

In [0]:
del data_train

In [0]:
del data

In [0]:
del word_index, nb_words, all_embs, list_tokenized_train, list_sentences_train, \
tokenizer

In [0]:
X.shape

(3516903, 200)

In [0]:
from keras import initializers, regularizers, constraints


def dot_product(x, kernel):
    """
    Wrapper for dot product operation, in order to be compatible with both
    Theano and Tensorflow
    Args:
        x (): input
        kernel (): weights
    Returns:
    """
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)
    

class AttentionWithContext(Layer):
    """
    initially taken from: https://gist.github.com/cbaziotis/7ef97ccf71cbc14366835198c09809d2
    Attention operation, with a context/query vector, for temporal data.
    Supports Masking.
    Follows the work of Yang et al. [https://www.cs.cmu.edu/~diyiy/docs/naacl16.pdf]
    "Hierarchical Attention Networks for Document Classification"
    by using a context vector to assist the attention
    # Input shape
        3D tensor with shape: `(samples, steps, features)`.
    # Output shape
        2D tensor with shape: `(samples, features)`.
    How to use:
    Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
    The dimensions are inferred based on the output shape of the RNN.
    Note: The layer has been tested with Keras 2.0.6
    Example:
        model.add(LSTM(64, return_sequences=True))
        model.add(AttentionWithContext())
        # next add a Dense layer (for classification/regression) or whatever...
    """

    def __init__(self, return_coefficients=False,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.return_coefficients = return_coefficients
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)

        self.u = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)

        super(AttentionWithContext, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        uit = dot_product(x, self.W)

        if self.bias:
            uit += self.b

        uit = K.tanh(uit)
        ait = dot_product(uit, self.u)

        a = K.exp(ait)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a

        if self.return_coefficients:
            return [K.sum(weighted_input, axis=1), a]
        else:
            return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        if self.return_coefficients:
            return [(input_shape[0], input_shape[-1]), (input_shape[0], input_shape[-1], 1)]
        else:
            return input_shape[0], input_shape[-1]

In [0]:
def auc_roc(y_true, y_pred):
    # any tensorflow metric
    value, update_op = tf.contrib.metrics.streaming_auc(y_pred, y_true)

    # find all variables created for this metric
    metric_vars = [i for i in tf.local_variables() if 'auc_roc' in i.name.split('/')[1]]

    # Add metric variables to GLOBAL_VARIABLES collection.
    # They will be initialized for new session.
    for v in metric_vars:
        tf.add_to_collection(tf.GraphKeys.GLOBAL_VARIABLES, v)

    # force to update metric values
    with tf.control_dependencies([update_op]):
        value = tf.identity(value)
        return value

In [0]:
reset_tf_session()

In [0]:
reset_tf_session()
INIT_LR = 5e-4

input_layer = Input((maxlen,), name = 'comment_text')
embedding_layer = Embedding(max_features, embed_size, input_length=maxlen, 
                            weights=[embedding_matrix], 
                            trainable = True)(input_layer)

embedding_layer = SpatialDropout1D(0.3)(embedding_layer)

lstm_1 = Bidirectional(CuDNNLSTM(128, return_sequences=True))(embedding_layer)
lstm_1 = Dropout(0.2)(lstm_1)

lstm_2 = Bidirectional(CuDNNLSTM(128, return_sequences=True))(lstm_1)
lstm_2 = Dropout(0.2)(lstm_2)

lstm_2 = add([lstm_2, lstm_1])

lstm_3 = Bidirectional(CuDNNLSTM(64, return_sequences=True))(lstm_2)
lstm_3 = Dropout(0.2)(lstm_3)

attention = AttentionWithContext()(lstm_3)

ffn_1 = Dense(32, activation="relu", kernel_initializer=he_normal())(attention)
ffn_1 = Dropout(0.3)(ffn_1)

output_layer = Dense(1, activation="sigmoid")(ffn_1)

model = Model(inputs=input_layer, outputs=output_layer)
model.compile(loss='binary_crossentropy',
                  optimizer=Adam(clipvalue=1, clipnorm=1),
                  metrics=[auc_roc])
print(model.summary())

def lr_scheduler(epoch):
  return INIT_LR * 0.8 ** epoch

lr = LearningRateScheduler(lr_scheduler)
    
early_stop = EarlyStopping(monitor='val_loss',
                           patience=4,
                           verbose=1,
                           min_delta=1e-4)


history = model.fit(X, y, batch_size=2048, epochs = 20, 
                    validation_split = 0.1, verbose = 1, callbacks = [lr, early_stop], shuffle=True)






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please switch to tf.metrics.auc. Note that the order of the labels and predictions arguments has been switched.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to         

In [0]:
prediction_df = pd.read_csv('random_prediction.csv')

In [0]:
prediction_df.Probability = model.predict(x_train_pad[3516903:])

In [0]:
prediction_df.to_csv("submission.csv", index=False)
! kaggle competitions submit -c iad-deep-learning-sentiment -m 'Shikunov Nikolay IAD2' -f submission.csv

100% 6.99M/6.99M [00:02<00:00, 2.85MB/s]
Successfully submitted to Sentiment analysis

In [0]:
! pip install transformers > /dev/null

In [0]:
from transformers import  BertTokenizer, BertForSequenceClassification

In [0]:
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

100%|██████████| 213450/213450 [00:00<00:00, 1089299.05B/s]


In [0]:
tokenizer.convert_tokens_to_ids(tokenizer.tokenize("This is my life no more words"))

[1188, 1110, 1139, 1297, 1185, 1167, 1734]

In [0]:
bert = 